In [1]:
import numpy as np
import sys 


from sympy import symbols, simplify, derive_by_array
from scipy.integrate import solve_ivp
from xLSINDy_sp_friction import *
from sympy.physics.mechanics import *
from sympy import *
from Data_generator_py_friction import image_process
import sympy
import torch
import sys
import HLsearch as HL
import example_pendulum
import time

In [ ]:
save = False
environment = "server"
sample_size = 10
device = 'cuda:7'

In [ ]:
sys.path.append(r'../../../HLsearch/')
#Saving Directory
params = {}
params['adding_noise'] = False
params['noise_type'] = 'image_noise'
params['noiselevel'] = 1e-1
params['changing_length'] = False
if environment == 'laptop':
    root_dir =R'C:\Users\87106\OneDrive\sindy\progress'
elif environment == 'desktop':
    root_dir = R'E:\OneDrive\sindy\progress'
elif environment == 'server':
    root_dir = R'/mnt/ssd1/stilrmy/Autoencoder-conservtive_expression'
x,dx,ddx = image_process(sample_size,params)
X = []
Xdot = []
for i in range(len(x)):
    temp_list = [float(x[i]),float(dx[i])]
    X.append(temp_list)
    temp_list = [float(dx[i]),float(ddx[i])]
    Xdot.append(temp_list)
X = np.vstack(X)
print(X.shape)
Xdot = np.vstack(Xdot)

sample version: c=-0.14
type of t:  float64
(5000, 2)


In [ ]:
states_dim = 2
states = ()
states_dot = ()
for i in range(states_dim):
    if(i<states_dim//2):
        states = states + (symbols('x{}'.format(i)),)
        states_dot = states_dot + (symbols('x{}_t'.format(i)),)
    else:
        states = states + (symbols('x{}_t'.format(i-states_dim//2)),)
        states_dot = states_dot + (symbols('x{}_tt'.format(i-states_dim//2)),)
print('states are:',states)
print('states derivatives are: ', states_dot)

states are: (x0, x0_t)
states derivatives are:  (x0_t, x0_tt)


In [ ]:
#Turn from sympy to str
states_sym = states
states_dot_sym = states_dot
states = list(str(descr) for descr in states)
states_dot = list(str(descr) for descr in states_dot)

In [ ]:
#build function expression for the library in str
expr= HL.buildFunctionExpressions(2,states_dim,states,use_sine=True)
#expr=['x0', 'x0_t', 'sin(x0)', 'cos(x0)', 'x0**2', 'x0*x0_t', 'x0_t**2', 'x0*sin(x0)', 'x0_t*sin(x0)', 'sin(x0)**2', 'x0*cos(x0)', 'x0_t*cos(x0)', 'sin(x0)*cos(x0)', 'cos(x0)**2']
"a list of candidate function"
print(expr)
expr.pop(5)

['x0', 'x0_t', 'sin(x0)', 'cos(x0)', 'x0**2', 'x0*x0_t', 'x0_t**2', 'x0*sin(x0)', 'x0_t*sin(x0)', 'sin(x0)**2', 'x0*cos(x0)', 'x0_t*cos(x0)', 'sin(x0)*cos(x0)', 'cos(x0)**2']


'x0*x0_t'

In [ ]:
Zeta, Eta, Delta,Gamma = LagrangianLibraryTensor(X,Xdot,expr,states,states_dot,device,scaling=True)
Eta = Eta.to(device)
Zeta = Zeta.to(device)
Delta = Delta.to(device)
Gamma = Gamma.to(device)

In [ ]:
mask = torch.ones(len(expr),device=device)
xi_L = torch.ones(len(expr), device=device).data.uniform_(-10,10)
prevxi_L = xi_L.clone().detach()
c = torch.ones(1,device=device)*-1
prec = c.clone().detach()

In [ ]:
def loss(pred, targ):
    loss = torch.mean((pred - targ)**2) 
    return loss 

In [ ]:
def clip(w, alpha):
    clipped = torch.minimum(w,alpha)
    clipped = torch.maximum(clipped,-alpha)
    return clipped

In [ ]:
def proxL1norm(w_hat, alpha):
    if(torch.is_tensor(alpha)==False):
        alpha = torch.tensor(alpha)
    w = w_hat - clip(w_hat,alpha)
    return w

In [ ]:
def training_loop(coef,c, prevcoef,prec, Zeta, Eta, Delta,Gamma,xdot, bs, lr, lam, momentum = True, beta1=0.9, beta2=0.999, eps=1e-8):
    loss_list = []
    tl = xdot.shape[0]
    n = xdot.shape[1]

    v = coef.clone().detach().to(device).requires_grad_(True)
    prev = prevcoef.clone().detach().to(device).requires_grad_(True)
    prec = prec.clone().detach().to(device).requires_grad_(True)
    
    # Initialize moving averages for Adam
    m_v = torch.zeros_like(v)
    m_c = torch.zeros_like(c)
    v_v = torch.zeros_like(v)
    v_c = torch.zeros_like(c)

    for i in range(tl//bs):
        vhat = v.requires_grad_(True).clone().detach().requires_grad_(True)
        chat = c.requires_grad_(True).clone().detach().requires_grad_(True)
        prev = v
        prec = c
        
        zeta = Zeta[:,i*bs:(i+1)*bs]
        eta = Eta[:,i*bs:(i+1)*bs]
        delta = Delta[:,i*bs:(i+1)*bs]
        gamma = Gamma[:,i*bs:(i+1)*bs]
        x_t = xdot[i*bs:(i+1)*bs,:]

        q_tt_pred = lagrangianforward(vhat,chat,zeta,eta,delta,gamma,x_t,device)
        q_tt_true = xdot[i*bs:(i+1)*bs,n//2:].T
        q_tt_true = torch.from_numpy(q_tt_true).to(device).float()
        lossval = loss(q_tt_pred, q_tt_true)
        lossval.requires_grad_(True)
        
        lossval.backward()

        with torch.no_grad():
            # Update moving averages
            m_v = beta1 * m_v + (1 - beta1) * vhat.grad
            v_v = beta2 * v_v + (1 - beta2) * (vhat.grad ** 2)
            m_c = beta1 * m_c + (1 - beta1) * chat.grad
            v_c = beta2 * v_c + (1 - beta2) * (chat.grad ** 2)

            # Compute bias-corrected moving averages
            m_v_hat = m_v / (1 - beta1 ** (i + 1))
            v_v_hat = v_v / (1 - beta2 ** (i + 1))
            m_c_hat = m_c / (1 - beta1 ** (i + 1))
            v_c_hat = v_c / (1 - beta2 ** (i + 1))

            # Update parameters
            c = chat - lr * m_c_hat / (torch.sqrt(v_c_hat) + eps)
            c = proxL1norm(c, lr * lam)
            v = vhat - lr * m_v_hat / (torch.sqrt(v_v_hat) + eps)
            v = proxL1norm(v, lr * lam)
        loss_list.append(lossval.item())
    return v,prev,c,prec,torch.tensor(loss_list).mean().item()


In [ ]:
Epoch = 100
i = 1
lr = 1e-3
lam = 0.1
temp = 1000
while(i<=Epoch):
    xi_L , prevxi_L,c,prec, lossitem= training_loop(xi_L,c,prevxi_L,prec,Zeta,Eta,Delta,Gamma,Xdot,128,lr=lr,lam=lam)
    if i %20 == 0:
        print("\n")
        print("Stage 1")
        print("Epoch "+str(i) + "/" + str(Epoch))
        print("Learning rate : ", lr)
        print("Average loss : " , lossitem)
    if(temp <=5e-3):
        break
    if(temp <=1e-1):
        lr = 1e-5
    temp = lossitem
    i+=1



Stage 1
Epoch 20/100
Learning rate :  0.001
Average loss :  32.938602447509766


Stage 1
Epoch 40/100
Learning rate :  0.001
Average loss :  30.137357711791992


Stage 1
Epoch 60/100
Learning rate :  0.001
Average loss :  26.571229934692383


Stage 1
Epoch 80/100
Learning rate :  0.001
Average loss :  21.84833335876465


Stage 1
Epoch 100/100
Learning rate :  0.001
Average loss :  15.776118278503418


In [ ]:
threshold = 1e-2
surv_index = ((torch.abs(xi_L) >= threshold)).nonzero(as_tuple=True)[0].detach().cpu().numpy()
expr = np.array(expr)[surv_index].tolist()

xi_L =xi_L[surv_index].clone().detach().requires_grad_(True)
prevxi_L = xi_L.clone().detach()
mask = torch.ones(len(expr),device=device)

## obtaining analytical model
xi_Lcpu = np.around(xi_L.detach().cpu().numpy(),decimals=2)
L = HL.generateExpression(xi_Lcpu,expr)
print(len(xi_L))
print(L)

13
-1.16*x0 + 0.29*x0_t + -8.45*sin(x0) + 12.25*cos(x0) + 5.7*x0**2 + 2.72*x0_t**2 + -5.26*x0*sin(x0) + 7.73*x0_t*sin(x0) + -8.81*sin(x0)**2 + 8.7*x0*cos(x0) + -6.91*x0_t*cos(x0) + -7.31*sin(x0)*cos(x0) + 1.57*cos(x0)**2 + 


In [ ]:
print(c)

tensor([2.2464], device='cuda:7')


In [ ]:
momentum = True
lr_initial = 1e-3
decay_rate = 0.01
for stage in range(1000):
    lr = lr_initial * np.exp(-decay_rate * stage)
    #Redefine computation after thresholding
    Zeta, Eta, Delta, Gamma = LagrangianLibraryTensor(X,Xdot,expr,states,states_dot,device,scaling=False)
    Eta = Eta.to(device)
    Zeta = Zeta.to(device)
    Delta = Delta.to(device)
    Gamma = Gamma.to(device)
    #Training
    Epoch = 200
    i = 1
    threshold = 2e-1
    if len(xi_L) == 2:
        #c_training = True
        threshold = 0
        momentum = True
    if(stage==1):
        lam = 0
    else:
        lam = 0.1
    temp = 1000
    while(i<=Epoch):   
        xi_L , prevxi_L,c,prec, lossitem= training_loop(xi_L,c,prevxi_L,prec,Zeta,Eta,Delta,Gamma,Xdot,128,lr=lr,lam=lam,momentum=momentum)
        if i %50 == 0:
            print("\n")
            print("Stage ",stage)
            print("Epoch "+str(i) + "/" + str(Epoch))
            print("Learning rate : ", lr)
            print("Average loss : " , lossitem)
        temp = lossitem

        i+=1
    
    ## Thresholding small indices ##

    surv_index = ((torch.abs(xi_L) >= threshold)).nonzero(as_tuple=True)[0].detach().cpu().numpy()
    expr = np.array(expr)[surv_index].tolist()

    xi_L =xi_L[surv_index].clone().detach().requires_grad_(True)
    prevxi_L = xi_L.clone().detach()
    mask = torch.ones(len(expr),device=device)

    ## obtaining analytical model
    xi_Lcpu = np.around(xi_L.detach().cpu().numpy(),decimals=3)
    c_cpu = c.detach().cpu().numpy()
    L = HL.generateExpression(xi_Lcpu,expr)
    print("expression length:\t",len(xi_L))
    print("Result stage " + str(stage+2) + ":" , L)
    print('exp({}*t)'.format(c_cpu))



Stage  0
Epoch 50/200
Learning rate :  0.001
Average loss :  7.856019692553673e-06


Stage  0
Epoch 100/200
Learning rate :  0.001
Average loss :  5.560499630519189e-06


Stage  0
Epoch 150/200
Learning rate :  0.001
Average loss :  1.4514201211568434e-05


Stage  0
Epoch 200/200
Learning rate :  0.001
Average loss :  7.868423381296452e-06
expression length:	 5
Result stage 2: -1.815*sin(x0) + 24.936*cos(x0) + 1.415*x0_t**2 + 20.278*x0_t*sin(x0) + -13.003*x0_t*cos(x0) + 
exp([0.13982226]*t)


Stage  1
Epoch 50/200
Learning rate :  0.000990049833749168
Average loss :  6.285237759584561e-06


Stage  1
Epoch 100/200
Learning rate :  0.000990049833749168
Average loss :  6.5794506554084364e-06


Stage  1
Epoch 150/200
Learning rate :  0.000990049833749168
Average loss :  5.330814474291401e-06


Stage  1
Epoch 200/200
Learning rate :  0.000990049833749168
Average loss :  1.097550102713285e-05
expression length:	 5
Result stage 3: -1.818*sin(x0) + 24.938*cos(x0) + 1.416*x0_t**2 + 20.268*x0_